### 1.2.2 BeautifulSoup 실행
---
- BeautifulSoup 라이브러리에서 가장 널리 쓰이는 객체는 물론 BeautifulSoup 객체이다.

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com/pages/page1.html')

# HTML 콘텐츠 얻기
bs = BeautifulSoup(html.read(), 'html.parser')
print(bs.h1)

<h1>An Interesting Title</h1>


- 객체를 만들 때는 두 가지 매개변수가 들어간다.
- `bs = BeautifulSoup(html.read(), 'html.paser')`
- `html.read` 는 이 객체의 근간이 되는 HMTL 텍스트이고,
- `html.parser` 는 BeautifulSoup가 객체를 만들 때 쓰는 구문 분석기인데, 이 구문 분석기는 우리가 직접 지정할 수 있다
  - `html.parser` 는 파이썬3와 함께 설치되므로 따로 설치할 필요는 없다.
  - lxml(http://lxml.de/parsing.html)도 널리 쓰이는 구문 분석기이다.
    - `pip3 install lxml`
    - `bs = BeautifulSoup(html.read(), 'lxml')`
    - lxml은 형식을 정확히 지키지 않은 '지저분한' HTML 코드를 분석할 때 html.parser보다 좀 더 낫다.
    - lxml은 닫히지 않은 태그, 계층 구조가 잘못된 태그, <head>나 <body> 태그가 없는 등의 문제에서 일일히 멈추지 않고 그 문제를 수정한다.
    - lxml은 html.parser에 비해 조금 더 빠르기도 하지만, 웹 스크래핑 분야에서는 네트워크 속도가 가장 큰 병목일 때가 대부분이므로 속도가 빠른 것은 별다른 장점이라 하기 어렵다.
  - html5lib도 널리 쓰이는 HTMl 구문 분석기이다.
    - lxml과 마찬가지로 html5lib도 잘못 만들어진 HTML을 수정하고 구문 분석을 시도하는데, lxml보다 더 다양한 에러를 수정할 수 있다.
    - 외부 프로그램이 있어야 동작할 수 있는 것도 마찬가지이며 lxml이나 html.parser보다 조금 느리다는 단점이 있긴 하지만, 잘못 만들어진 HTML이나 손으로 쓴 HTML을 분석해야 한다면 html5lib도 좋은 선택이다.

### 1.2.3 신뢰할 수 있는 연결과 예외 처리
---
- 스크레이퍼의 임포트 문 다음 행을 살펴보고 예외를 어떻게 처리할지 생각해본다.

In [4]:
html = urlopen('http://www.pythonscraping.com/pages/page1.html')

- 이 행에서 문제가 생길 수 있는 부분은 크게 두 가지이다.
  - 페이지를 찾을 수 없거나, URL 해석에서 에러가 생긴 경우
  - 서버를 찾을 수 없는 경우
- 첫 번째 상황에서는 HTTP 에러가 반환될 것이다.
- 이 HTTP 에러는 '404 Page Not Found', '500 Internal Server Error' 등이다.
- 이런 모든 경우에 urlopen 함수는 범용 예외인 HTTPError를 일으킨다.

- 이 예외는 다음과 같이 처리한다.

In [ ]:
from urllib.request import urlopen
from urllib.request import HTTPError

try:
    html = urlopen('http://www.pythonscraping.com/pages/page1.html')
except HTTPError as e:
    print(e)
    # None을 반환하거나, break 문을 실행하거나, 기타 다른 방법으로 처리함
else:
    # 프로그램을 계속 실행

- 이제 HTTP 에러 코드가 반환되면 프로그램은 에러를 출력하고 else 문은 실행하지 않는다.
- 서버를 전혀 찾을 수 없을 때, 예를 들어 특정 사이트가 다운됐거나 URL에 오타가 있을 때 urlopen은 URLError 예외를 일으킨다.
- HTTP 상태 코드를 얻으려면 원격 서버에 접속할 수 있어야 하는데 그 서버에 전혀 접속할 수 없으므로 HTTPError는 일어날 리 없는 상황이며,
- 그보다 한 단계 더 심각한 URLError를 캐치해야 한다.


- URLError도 캐치하려면 다음과 같은 코드를 사용한다.

In [5]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

try:
    html = urlopen('http://www.pythonscraping.com/pages/page1.html')
except HTTPError as e:
    print(e)
except URLError as e:
    print("The server could not be found!")
else:
    print("It worked!")

It worked!


- 물론 페이지를 서버에서 성공적으로 가져왔어도 페이지 콘텐츠가 예상과 다를 수 있다.
- BeautifulSoup 객체에 들어있는 태그에 접근할 때마다 그 태그가 실제 존재하는지 체크하는 편이 좋다.
- 존재하지 않는 태그에 접근을 시도하면 BeautifulSoup는 None 객체를 반환한다.
- 문제는 None 객체 자체에 태그가 있다고 가정하고 그 태그에 접근하려 하면 AttributeError가 발생한다.

- 다음 코드에서 nonExistentTag는 존재한다고 가정하는 태그이며 실제 BeautifulSoup 함수 이름은 아니다.

In [6]:
print(bs.nonExistentTag)

None


/Users/yangws/Library/Python/3.8/lib/python/site-packages/bs4/element.py:1524: UserWarning: .nonExistentTag is deprecated, use .find("nonExistent") instead. If you really were looking for a tag called nonExistentTag, use .find("nonExistentTag")
  warnings.warn(


- 위 코드는 None 객체를 반환한다. 
- None 객체는 처리하거나 체크할 때 아무 문제도 없다.

- 문제는 다음 예제처럼 None이 반환될 수 있음을 무시하고 None 객체에 어떤 함수를 호출하는 경우이다.

In [7]:
print(bs.nonExistentTag.someTag)

/Users/yangws/Library/Python/3.8/lib/python/site-packages/bs4/element.py:1524: UserWarning: .nonExistentTag is deprecated, use .find("nonExistent") instead. If you really were looking for a tag called nonExistentTag, use .find("nonExistentTag")
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'someTag'

- 위 코드는 다음과 같은 예외를 일으킨다.
- `AttributeError: 'NoneType' object has no attribute 'someTag'`

- 이런 두 가지 상황에 대응하는 가장 쉬운 방법은 명시적으로 체크하는 것이다.

In [ ]:
try:
    badContent = bs.nonExistentTag.anotherTag
except AttributeError as e:
    print("Tag was not found")
else:
    if badContent == None:
        print("Tag ws not found")
    else:
        print(badContent)

- 이렇게 가능한 에러를 모두 체크하고 처리하는 일이 처음에는 지겨워 보일 수 있지만, 코드를 조금만 수정하면 좀 더 쉽게 읽을 수 있다.

---
- 다음 코드는 같은 스크레이퍼를 조금 다르게 쓴 것이다.

In [9]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

def getTitle(url):
    
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    
    try:
        bs = BeautifulSoup(html.read(), 'html.parser')
        title = bs.body.h1
    except AttributeError as e:
        return None

    return title

title = getTitle('http://www.pythonscraping.com/pages/page1.html')

if title == None:
    print("Title could not be found")
else:
    print(title)

<h1>An Interesting Title</h1>


<p>이 예제에서는 페이지 타이틀을 반환하거나, 어떤 문제가 있으면 None 객체를 반환하는 getTitle 함수를 만든다.<br>
getTitle 내부에서는 이전 예제와 마찬가지로 HTTPError를 체크하고 BeautifulSoup 행 두 개를 try 문으로 캡슐화한다.<br>
이 두 행 중 어느 행이라도 AttributeError를 일으킬 수 있다. 예를 들어 서버가 존재하지 않으면 html은 None 객체이고 html.read()가 AttributeError를 일으킬 것이다.<br>
try 문 하나에 원하는 만큼 여러 행을 넣을 수도 있고, AttributeError를 일으킬 수 있는 별도의 함수도 어느 시점에서는 호출할 수 있다.
</p>

- 스크레이퍼를 만들 때는 코드의 전반적 패턴에 대해 생각해야 예외도 처리하고 읽기도 쉽게 만들 수 있다.
- 코드를 많이 재사용하고 싶을 때는 getStieHTML이나 getTitle 같은 범용 함수를 만들고 여기에 예외 처리를 철저하게 만들어두면 빠르고 믿을 수 있는 웹 스크레이퍼를 쉽게 만들 수 있다.